In [84]:
import os
import sys
import math
import pandas
import numpy as np
from PIL import Image
import tensorflow as tf
from matplotlib import pyplot as plt
from scipy.misc import imread, imsave, imresize

PATH_TO_CKPT = 'exported_graphs/frozen_inference_graph.pb'
# data_dir = "GTSDBxGTSRB"
data_dir = "GTSDB"
tf_models_dir = "/home/matech/projects/tf_models"

In [85]:
with open("train.record.namestxt") as f:
    file_list = f.read().splitlines()

In [86]:
labels = pandas.read_csv(os.path.join(data_dir, "gt.txt"), sep=";", header=None, 
                         names=("file_name", "x_min", "y_min", "x_max", "y_max", "label"))
# with open("train.record.namestxt") as f:
#     lines = f.read().splitlines()
data_to_check = labels#[labels["file_name"].isin(lines)]
n_all_images = len(data_to_check["file_name"].unique())

In [87]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [88]:
pic_shape = (320, 544, 3)
def load_image_into_numpy_array(file_names):
    xs = []
    for file_name in file_names:
#         image = imresize(imread(os.path.join(data_dir, file_name.replace(".ppm", ".jpg"))), pic_shape)
        image = imresize(imread(os.path.join(data_dir, file_name)), pic_shape)
        xs.append(image) # np.append(xs, image_expanded, axis=0)
    return np.array(xs)

In [89]:
file_names = file_list #np.random.choice(data_to_check["file_name"].unique().tolist(), min(800, n_all_images))
ys  = []
for file_name in file_names:
    ys.append((data_to_check[data_to_check["file_name"] == file_name]["label"] + 1).astype(int).unique().tolist())
# xs = load_image_into_numpy_array(data_to_check["file_name"].unique())

In [90]:
import time
batch_size = 6
n_images = len(ys)
accs = pandas.DataFrame(columns=["acc"], index=data_to_check["file_name"].unique())
results = {}
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        scores = []
        classes = []
        for i in range(math.ceil(n_images/batch_size)):
#             print(i/math.ceil(n_images/batch_size))
            start = i*batch_size
            batch_file_names = file_names[start:min(n_images,start+batch_size)]
            xs = load_image_into_numpy_array(batch_file_names)
            start = time.time()
            (ss, cs) = sess.run([detection_scores, detection_classes], 
                              feed_dict={image_tensor: xs})
            end = time.time()
            print(end-start)
            for s, c in zip(ss, cs):
                scores.append(s)
                classes.append(c)

0.8407177925109863
0.14804625511169434
0.1598970890045166
0.14768433570861816
0.16559529304504395
0.15615129470825195
0.15861845016479492
0.15379047393798828
0.15496158599853516
0.14969778060913086
0.17400431632995605
0.14981961250305176
0.1685163974761963
0.14534306526184082
0.17475128173828125
0.15217351913452148
0.1690206527709961
0.1482248306274414
0.15779495239257812
0.1446840763092041
0.16623616218566895
0.14583945274353027
0.14745044708251953
0.14813852310180664
0.14474129676818848
0.15394330024719238
0.17284107208251953
0.14422369003295898
0.14600062370300293
0.17301082611083984
0.17167186737060547
0.1602494716644287
0.15847063064575195
0.17820978164672852
0.14562392234802246
0.14563441276550293
0.16998291015625
0.14786934852600098
0.17051076889038086
0.14806342124938965
0.14718198776245117
0.15010690689086914
0.15691852569580078
0.17043137550354004
0.15372133255004883
0.14795255661010742
0.15153098106384277
0.15254902839660645
0.14708352088928223
0.1485447883605957
0.146425485

In [91]:
accs = []
for i in range(n_images):
    score = scores[i]
    classe = classes[i]
    res = pandas.DataFrame(data={"score": score, "class": classe})
    predicted_labels = res[res["score"] > 0.25]["class"].astype(int)
    gt_labels = pandas.Series(ys[i])
    results[file_name] = (predicted_labels, gt_labels, score, classe)
    correct_pred_labels = predicted_labels[predicted_labels.isin(gt_labels)].unique()
    all_labels_involved = predicted_labels.append(gt_labels).unique()
    if len(all_labels_involved) != 0:
        acc = len(correct_pred_labels)/len(all_labels_involved)
        accs.append(acc)
    else:
        print("No label:" + file_name)

In [92]:
print(np.mean(accs))
perfects = list(x for x in accs if x > 0.99)
print(len(perfects)/len(accs))

0.476891166423
0.38799414348462663
